### Kiran Reddy
#### CM3070 Final Year Project 

### Introduction 

This is my Final Year Project Prototype for the endterm of the CM3070 module. Throughout this project, 
I have followed the universal Machine Learning workflow. 

The dataset used is the MBTI (Multi Briggs Personality Type Dataset) from Kaggle. The Myers-Briggs Person Dataset categorises an individual into one of the 16 different types across 4 main axes. Figure 1 is an example image on how the dataset looks like. There are in total 16 different personalities among the four different axes. 

The main motivation to take on this topic is that the use of social media has become extremely prevalent in today's society, and top companies have leveraged on the power of ML and AI to use data acquired from various platforms to improve their business. 

I am using the template based on CM3015 Machine Learning and Neural Networks template. It will follow machine learning with deep learning from a public dataset template.

###### 1. Stating the problem statement and gathering a dataset

The problem in this project is to accruately categorise each of the last 50 things they have posted into 16 different personality types across 4 different axis. (Introversion, Intuition, Thinking, Jugding). This dataset contains 86000 rows of data and two columns, the type and texts. 

###### 2. Choosing a measure of success 

The measure of success in this project will be the accuracy at the end. In other words, the more accurate the model can classify the images in the correct category, the more successful this project is. I have first started the implementation of the project by doing some pre processing and then splitting the dataset into the standard 60-40 ratio. After which I have implemented 4 different models and compared each of their accuracies. 

Again after that, I prepared the data this time with a 70-30 ratio and also implemented the models. This time the models show a much better result. After this, I also implement the hyper parameter training on the dataset. 

###### 3. Deciding on the evaluation protocole

The evaluation protocol that I chose is by maintaining a hold-out validation (train, test) set which is the way to go when there is plenty of data. First I follow the 60-40 and then the 70-30 ratio dataset. While training the models, I made sure to train them seperately for each MBTI personality type, to get better results. 

###### 4. Preparing the data

In order to prepare my data, I will need to format my data by removing multiple punctuation, non words, stripping of punctuation, removing links and converting all the posts to lower case. 

Pre - Processing Steps:
Removing of links
I do this by using the lambda function and reading if there are any lines of code containing the https,?,,/,etc. If there are, I would simply replace them by spaces

Remaining the end of sentence characters such as the '?', '/','' Again I do this by reading through the texts to see if any of these characters are present.

Removal of fullstops I read through the texts to see if any of the fullstops are present and will remove it to enable more accurate results and fullstops or non prominent words wont add to the training and testing of my data.

Removal of non words

Converting the texts to all lowercase This is to better read and train the dataset to enable more accurate results
Similarly removal of too short or too long words. I find that these words dont add too much of a value and are redundant and thus have removed words which are maximum of 3 letters and words that are in between 30 - 1000 letters.

Removing the personality words in the texts This is because if these words are in the texts, then it would influence the prediction and may interfere with the model

###### 5. Developing a model 

Three points that I kept in my mind while developing the model was, the last-layer activation, loss function, and the optimization configuration. The last layer activation is to make sure the the constraints on the output of the previous layer. The loss function will define the feedback signal used for learning. And lastly, the optimization configuration which determines how learning proceeds, based on the loss function. After running through the models and implementing them seperately for the 60-40 ration and 70-30 ratio, I also implemented the hyperparameter training on the model to get better results but only managed to improve the results of one particular personality type. 

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


#### Importing the relevant libraries consiting of data analytics, data visualization, text processing, training and evaluations model

Note: These libraries have not been installed in order because I added in new lines of codes in between the project

In [2]:
import pandas as pd
import numpy as np
from numpy import asarray
import pickle as pkl
from scipy import sparse
import re
import seaborn as sns
import wordcloud
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from sklearn.preprocessing import LabelEncoder
from numpy import loadtxt
from numpy import savetxt
import itertools
import string
import collections
from collections import Counter
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.classify import NaiveBayesClassifier

from nltk import word_tokenize
from xgboost import XGBClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.manifold import TSNE
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
import sklearn.cluster as cluster
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report


import warnings
warnings.filterwarnings("ignore")

### Preparing the Data

##### Importing the dataset, link to the datatset: https://www.kaggle.com/datasnaek/mbti-type

In [3]:
#loading dataset
mydata = pd.read_csv("/Users/Kiranreddy/Desktop/FYP/mbti.csv")

# Going to show the last few lines of my dataset, basically the last 5 records of the csv files
mydata.tail()

,type,posts
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...
8674,INFP,'It has been too long since I have been on per...


### Pre Processing of Data

##### In order to prepare my data, I will need to format my data by removing multiple punctuation, non words, stripping of punctuation, removing links and converting all the posts to lower case. 

Pre - Processing Steps: 

1. Removing of links 

I do this by using the lambda function and reading if there are any lines of code containing the https,?,\,/,etc.
If there are, I would simply replace them by spaces/ 

2. Remaining the end of sentence characters such as the '?', '/','\'
Again I do this by reading through the texts to see if any of these characters are present. 

3. Removal of fullstops 
I read through the texts to see if any of the fullstops are present and will remove it to enable more accurate results and fullstops or non prominent words wont add to the training and testing of my data. 

4. Removal of non words 

5. Converting the texts to all lowercase 
This is to better read and train the dataset to enable more accurate results 

6. Similarly removal of too short or too long words. 
I find that these words dont add too much of a value and are redundant and thus have removed words which are maximum of 3 letters and words that are in between 30 - 1000 letters. 

7. Removing the personality words in the texts 
This is because if these words are in the texts, then it would influence the prediction and may interfere with the model

In [4]:
def preprocess_method(df, remove_special=True):
    posts = df['posts'].copy()
    labels_type = df['type'].copy()

   
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[^\w\s]','',x))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'!', ' EOSTokenExs ', x + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'\?', ' EOSTokenQuest ', x + " "))
    if remove_special:
        pers_types = ['INFP' ,'INFJ', 'INTP', 'INTJ', 'ENTP', 'ENFP', 'ISTP' ,'ISFP' ,'ENTJ', 'ISTJ','ENFJ', 'ISFJ' ,'ESTP', 'ESFP' ,'ESFJ' ,'ESTJ']
        pers_types = [p.lower() for p in pers_types]
        p = re.compile("(" + "|".join(pers_types) + ")")
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[\.+]', ".",x))
    df["posts"] = df["posts"].apply(lambda x: x.lower())
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'\.', ' EOSTokenDot ', x + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x)) 
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'(\b\w{30,1000})?\b','',x))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'(\b\w{0,3})?\b','',x))
    
    return df

preprocess_df = preprocess_method(mydata)

#### Initialising the minimum words to be 16 and removing other texts that have less than 16

In [5]:
minimum_words = 16
print("Number of Posts in dataset before: ", len(preprocess_df)) 
preprocess_df["number words"] = preprocess_df["posts"].apply(lambda x: len(re.findall(r'\w+', x)))
preprocess_df = preprocess_df[preprocess_df["number words"] >= minimum_words]

print("Number of Posts in dataset after: ", len(preprocess_df))

Number of Posts in dataset before:  8675
Number of Posts in dataset after:  8426


#### As you can see, we have removed over 200 texts that have texts less than 16 words. Now we will be working with this new dataset after pre process. 

In [6]:
preprocess_df.head()

,type,posts,number words
0,INFJ,intj moments sportscenter plays pra...,329
1,ENTP,finding lack these posts very alarmingsex...,626
2,INTP,good course which know thats blessi...,209
3,INTJ,dear intp enjoyed conversation other es...,603
4,ENTJ,youre firedthats another silly misconception t...,312


#### Transforming the MBTI personality types for the target values into numerical form usinig the labelEncoder function 

In [7]:
encoder = LabelEncoder()
preprocess_df['Encoding Type'] = encoder.fit_transform(preprocess_df['type'])

targetY = preprocess_df['Encoding Type'] 

#### Similarly transforming the MBTI personality types for the train values into numerical form usinig the Vectorizer function 

In [8]:
vectorizer = CountVectorizer(stop_words='english') 
trainX =  vectorizer.fit_transform(preprocess_df["posts"])

### Splitting the training and test dataset into 60 - 40 ratio to see how the accuracies are for the models

In [9]:
X_train, X_test, y_train, y_test = train_test_split(trainX, targetY, test_size=0.4, stratify=targetY, random_state=41)
print ((X_train.shape),(y_train.shape),(X_test.shape),(y_test.shape))

(5055, 229133) (5055,) (3371, 229133) (3371,)


In [12]:
trainX.shape

(8426, 229133)

In [14]:
targetY.shape

(8426,)

### Starting to run my models on the dataset to see the accuriacies acheieved by each of them. 

The different models I will be using will be: 
    
     - KNN Classifier
     - Logisitic Regression
     - XG Boost 
     - SVM

#### Defining an array named accuracy_array so that this will help me to output when I compare the different models and I can store the different accuracies in this array

In [15]:
accuracy_array = {}

### KNN Model Classifier

In [16]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
# Using K value as 2 here 

modelknn = KNeighborsClassifier(n_neighbors = 2) 
modelknn.fit(X_train, y_train)

Y_results = modelknn.predict(X_test)
predic = [round(value) for value in Y_results]

# These will be the predictions by the KNN model
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['KNN Model'] = accuracy_value* 100.0
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

Accuracy: 16.14%


In [23]:
## This is for me to find the best value of K here, to improve the accuracy results
KValueList = []
# Between a value of 1 - 19, checking which gives the best result

ListOfScores = []
for j in range(1,19):
    knnModelTest = KNeighborsClassifier(n_neighbors = j) 
    knnModelTest.fit(X_train, y_train)
    KValueList.append(knnModelTest.score(X_test, y_test))

best_score = max(KValueList)*100

print("Best Result for KNN is {:.2f}%".format(best_score))

Best Result for KNN is 24.83%


### XG Boost

In [25]:
modelXGB = XGBClassifier()
modelXGB.fit(X_train,y_train)

Y_pred = modelXGB.predict(X_test)
predic = [round(value) for value in Y_pred]

# These will be the predictions by the mode
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['XG Boost Model'] = accuracy_value* 100.0
print(accuracy_score(y_train, modelXGB.predict(X_train)))
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

0.9992087042532146
Accuracy: 53.90%


### Logistic Regression

In [26]:
logisregress = LogisticRegression()
logisregress.fit(X_train, y_train)

Y_predict = logisregress.predict(X_test)
predic = [round(value) for value in Y_predict] 


# These will be the predictions by the mode
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['Logistic Regression Model'] = accuracy_value* 100.0
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

Accuracy: 50.76%


### SVM

In [27]:
from sklearn.svm import SVC

In [28]:
ModelSVM = SVC(random_state = 1)
ModelSVM.fit(X_train, y_train)

predictY = ModelSVM.predict(X_test)

predic = [round(value) for value in predictY]
# These will be the predictions by the mode
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['SVM Model'] = accuracy_value* 100.0
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

Accuracy: 48.47%


In [29]:
pd.DataFrame.from_dict(accuracy_array, orient='index', columns=['Accuracy in %'])

,Accuracy in %
KNN Model,16.137645
XG Boost Model,53.900920
Logistic Regression Model,50.756452
SVM Model,48.472263


## Splitting the training and test dataset into 70 - 30 ratio to see how the accuracies are for the models

In [30]:
X_train, X_test, y_train, y_test = train_test_split(trainX, targetY, test_size=0.33, stratify=targetY, random_state=42)
print ((X_train.shape),(y_train.shape),(X_test.shape),(y_test.shape))

(5645, 229133) (5645,) (2781, 229133) (2781,)


### KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [33]:
# Using K value as 2 here 
ModelKNN = KNeighborsClassifier(n_neighbors = 2) 
ModelKNN.fit(X_train, y_train)

predictY = ModelKNN.predict(X_test)
predic = [round(value) for value in predictY]

# These will be the predictions by the mode
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['KNN Model'] = accuracy_value* 100.0
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

Accuracy: 15.82%


In [34]:
# This is for me to find the best value of K here, to improve the accuracy results
KValueList = []
# Between a value of 1 - 19, checking which gives the best result

for j in range(1,19):
    knnModelTest = KNeighborsClassifier(n_neighbors = j) 
    knnModelTest.fit(X_train, y_train)
    KValueList.append(knnModelTest.score(X_test, y_test))
    

best_result = max(KValueList)*100

print("Best Result for KNN is {:.2f}%".format(best_result))

Best Result for KNN is 23.12%


### XG Boost

In [35]:
modelXGB = XGBClassifier()
modelXGB.fit(X_train,y_train)

Y_pred = modelXGB.predict(X_test)
predic = [round(value) for value in Y_pred]

# These will be the predictions by the mode
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['XG Boost Model'] = accuracy_value* 100.0
print(accuracy_score(y_train, modelXGB.predict(X_train)))
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

0.9969884853852967
Accuracy: 56.02%


### SVM

In [36]:
ModelSVM = SVC(random_state = 1)
ModelSVM.fit(X_train, y_train)

predictY = ModelSVM.predict(X_test)

predic = [round(value) for value in predictY]
# These will be the predictions by the mode
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['SVM Model'] = accuracy_value* 100.0
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

Accuracy: 49.62%


### Logistic Regression

In [37]:
logisregress = LogisticRegression()
logisregress.fit(X_train, y_train)

Y_predict = logisregress.predict(X_test)
predic = [round(value) for value in Y_predict]


# These will be the predictions by the mode
accuracy_value = accuracy_score(y_test, predic)
# Also storing the value of accuracy into the array
accuracy_array['Logistic Regression Model'] = accuracy_value* 100.0
print("Accuracy: %.2f%%" % (accuracy_value * 100.0))

Accuracy: 50.49%


In [38]:
pd.DataFrame.from_dict(accuracy_array, orient='index', columns=['Accuracy in %'])

,Accuracy in %
KNN Model,15.821647
XG Boost Model,56.023013
Logistic Regression Model,50.485437
SVM Model,49.622438


In [39]:
mydata = pd.read_csv("/Users/Kiranreddy/Desktop/FYP/mbti.csv")
mydata.tail()

,type,posts
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...
8674,INFP,'It has been too long since I have been on per...


### Phase 2 - Using just the four axes instead of all the 16 personality groups as it gave me very low accuracy results

### Pre Processing 

In [40]:
lemmat = WordNetLemmatizer()
# I am removing all the stop words that are in English
stop_words = stopwords.words("english")

In [42]:
# Removal of the personality type words from the texts and equating it to an array called MBTI array
MBTI_array = ['INFJ', 'ENTP','ESTP', 'INTP','ISFP', 'ISTP', 'ISFJ', 'ISTJ','ESFP',
              'INTJ', 'ESTJ','ENTJ', 'ENFJ', 'INFP', 'ENFP', 'ESFJ']
MBTI_array = [y.lower() for y in MBTI_array]

In [95]:
# Making the MBTI personality types into a numerical form, for example I = 0 and E = 1

personality_numerical = {'F':0,'N':0, 'J':0,'T':1,'S':1, 'I':0,'P':1,'E':1,}
personality_numerical_list = [{0:'J', 1:'P'},{0:'F', 1:'T'},{0:'I', 1:'E'}, {0:'N', 1:'S'}]

# converting the personality types of MBTI into Binary 
def convert(personality):
    return [personality_numerical[y] for y in personality]


#Reverse conversion from binary
def convertBack(personality):
    m = ""
    for i, y in enumerate(personality):
        m += personality_numerical_list[i][y]
    return m

listBinary_personality = np.array([convert(p) for p in mydata.type])
print("Binarized Version \n%s" % listBinary_personality)

Binarized Version 
[[0 0 0 0]]


In [81]:
def text_preprocessing(data, remove_stop_words=True, remove_mbti_profiles=True):
  personalityList = []
  textsList = []
  dataLength = len(data)
  j=0
  
  for entry in data.iterrows():
        

      texts = entry[1].posts
        
      garb = re.sub("[^a-zA-Z]", " ", texts)
      garb = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', texts)
      garb = re.sub(' +', ' ', texts).lower()
      garb = re.sub(r'([a-z])\1{2,}[\s|\w]*', '', texts)
      if remove_stop_words:
          garb = " ".join([lemmat.lemmatize(y) for y in garb.split(' ') if y not in stop_words])
      else:
          garb = " ".join([lemmat.lemmatize(y) for y in garb.split(' ')])
          
      #Remove MBTI personality words from posts
      if remove_mbti_profiles:
          for t in MBTI_array:
              garb = garb.replace(t,"")

      # transform mbti to binary vector
      type_labelized = convert(entry[1].type)
      personalityList.append(type_labelized)
      textsList.append(garb)
    
  textsList = np.array(textsList)
  personalityList = np.array(personalityList)
  return textsList, personalityList

textsList, personalityList  = text_preprocessing(mydata, remove_stop_words=True, remove_mbti_profiles=True)

#### Printing the number of columns and rows of the dataset, outputting the number of texts and number of personality groups

In [50]:
numberOfRows, numberOfColumns = personalityList.shape
print(f'Number of texts = {numberOfRows}  and Personality Groups = {numberOfColumns} ')

Number of texts = 8675  and Personality Groups = 4 


### Feature Engineering 

#### Now I am going to further process the dataset in order to remove words that are not relevant. I will do this by vectorizing the texts based on a certain margin. The margin that I have decided is between 12% to 75%. Any words in the dataset that do not occur as frequent as this percentage will be removed so only the releavnt words will be in place. This is also known as feature engineering. 

In [51]:
#Initializing a variable names CVZ (count Vectorizer)
CVZ = CountVectorizer(analyzer="word", 
                             max_features=1000,  
                             max_df=0.75,
                             min_df=0.12) 

print("Using CountVectorizer :")
number_of_X = CVZ.fit_transform(textsList)

names_of_features = list(enumerate(CVZ.get_feature_names()))
print("A few of the features:")
print(names_of_features[0:5])

tfi = TfidfTransformer()

print("\With TFIDF :")

print("Now the dataset size is as below")
tfi_X =  tfi.fit_transform(number_of_X).toarray()
print(tfi_X.shape)

Using CountVectorizer :
A few of the features:
[(0, '10'), (1, '100'), (2, '20'), (3, 'ability'), (4, 'able')]
\With TFIDF :
Now the dataset size is as below
(8675, 755)


#### Splitting into X and Y 

In [52]:
personality_type = [ "IE: Introversion (I) / Extroversion (E)", "NS: Intuition (N) / Sensing (S)", 
                   "FT: Feeling (F) / Thinking (T)", "JP: Judging (J) / Perceiving (P)"  ]

for x in range(len(personality_type)):
    print(personality_type[x])

IE: Introversion (I) / Extroversion (E)
NS: Intuition (N) / Sensing (S)
FT: Feeling (F) / Thinking (T)
JP: Judging (J) / Perceiving (P)


#### Training and Evaluating Models

In [ ]:
#### Initialising the posts that are in the TFI-DF format to a variable A

In [53]:
A = tfi_X

### Random Forest

#### In order to improve the accuracy rates, I have also implemented a method in which I will train each personality type seperately as you can see below

In [57]:
for i in range(len(personality_type)):
    
    M = personalityList[:,i]

    # Now we will split the data into test and train according to the 70-30% ratio
    
    X_train, X_test, y_train, y_test = train_test_split(A, M, test_size=0.33, random_state=7)

    TypeModel = RandomForestClassifier()
    TypeModel.fit(X_train, y_train)

    predictY= TypeModel.predict(X_test)
     
    results = [round(x) for x in predictY]
    scores = accuracy_score(y_test, results)
    
    print("%s The Accuracy is: %.2f%%" % (personality_type[i], scores * 100.0))

IE: Introversion (I) / Extroversion (E) The Accuracy is: 79.71%
NS: Intuition (N) / Sensing (S) The Accuracy is: 86.31%
FT: Feeling (F) / Thinking (T) The Accuracy is: 80.34%
JP: Judging (J) / Perceiving (P) The Accuracy is: 74.15%


### XG Boost

In [58]:
for i in range(len(personality_type)):
    
    M = personalityList[:,i]

    # Now we will split the data into test and train according to the 70-30% ratio
    
    X_train, X_test, y_train, y_test = train_test_split(A, M, test_size=0.33, random_state=7)

    TypeModel = XGBClassifier()
    TypeModel.fit(X_train, y_train)

    predictY= TypeModel.predict(X_test)
    
    # Checking against the results and the accuracies 
    results = [round(x) for x in predictY]
    scores = accuracy_score(y_test, results)
    
    print("%s The Accuracy is: %.2f%%" % (personality_type[i], scores * 100.0))

IE: Introversion (I) / Extroversion (E) The Accuracy is: 85.99%
NS: Intuition (N) / Sensing (S) The Accuracy is: 89.91%
FT: Feeling (F) / Thinking (T) The Accuracy is: 81.49%
JP: Judging (J) / Perceiving (P) The Accuracy is: 78.45%


### Logistic Regression

In [59]:
for i in range(len(personality_type)):
    
    M = personalityList[:,i]

    # Now we will split the data into test and train according to the 70-30% ratio
    
    X_train, X_test, y_train, y_test = train_test_split(A, M, test_size=0.33, random_state=7)

    TypeModel = LogisticRegression()
    TypeModel.fit(X_train, y_train)

    predictY= TypeModel.predict(X_test)
    
    # Checking against the results and the accuracies 
    results = [round(x) for x in predictY] 
    scores = accuracy_score(y_test, results)
    
    print("%s The Accuracy is: %.2f%%" % (personality_type[i], scores * 100.0))

IE: Introversion (I) / Extroversion (E) The Accuracy is: 85.30%
NS: Intuition (N) / Sensing (S) The Accuracy is: 88.47%
FT: Feeling (F) / Thinking (T) The Accuracy is: 84.00%
JP: Judging (J) / Perceiving (P) The Accuracy is: 78.76%


### SVM

In [60]:
for i in range(len(personality_type)):
    
    M = personalityList[:,i]

    # Now we will split the data into test and train according to my second implementation ratio of 70-30
    
    X_train, X_test, y_train, y_test = train_test_split(A, M, test_size=0.33, random_state=7)

    TypeModel = SVC(random_state = 1)
    TypeModel.fit(X_train, y_train)

    predictY= TypeModel.predict(X_test)
    
    # Checking against the results and the accuracies 
    results = [round(x) for x in predictY]
    scores = accuracy_score(y_test, results)
    
    print("%s The Accuracy is: %.2f%%" % (personality_type[i], scores * 100.0))

IE: Introversion (I) / Extroversion (E) The Accuracy is: 85.89%
NS: Intuition (N) / Sensing (S) The Accuracy is: 88.47%
FT: Feeling (F) / Thinking (T) The Accuracy is: 83.90%
JP: Judging (J) / Perceiving (P) The Accuracy is: 78.83%


#### We notice that XGBoost gives me the best results and have improved alot from the baseline models. 

Now I will try to implement hyperparameter tuning to increase the accuracies even more. 

### Hyper Parameter Tuning

In [66]:
hyperpar = {}
hyperpar['n_estimators'] = 199
hyperpar['max_depth'] = 3
hyperpar['nthread'] = 7
hyperpar['learning_rate'] = 0.1


for i in range(len(personality_type)):
    M = personalityList[:,i]

    # Splitting the dataset into the relevant train and test data following the same 70 30 ratio 
    seed = 13
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(A, M, test_size=test_size, random_state=seed)
    TypeModel = XGBClassifier(**hyperpar)
    TypeModel.fit(X_train, y_train)
    
    
    # Initialising the prediction values now 
    
    predictY = TypeModel.predict(X_test)
     # Checking against the results and the accuracies 
    results = [round(x) for x in predictY]
    scores = accuracy_score(y_test, results)
    print("%s The Accuracy is: %.2f%%" % (personality_type[i], scores * 100.0))

IE: Introversion (I) / Extroversion (E) The Accuracy is: 84.70%
NS: Intuition (N) / Sensing (S) The Accuracy is: 89.73%
FT: Feeling (F) / Thinking (T) The Accuracy is: 82.89%
JP: Judging (J) / Perceiving (P) The Accuracy is: 79.74%


### Testing the program with a sample short essay

In [88]:
testPost = """At St. Albans, especially in our later years, we are given the freedom to choose from a 
vast array of classes. Using this freedom, I have selected classes which have personal significance to me, 
regardless of difficulty or appearance on my transcript. However, from these classes, one holds an extraordinary 
amount of value to me. This course is A.P. (Omnibus) History, a combination of American and European history. 
There are several reasons for my great interest in this class. First, I am fascinated by the cyclical nature 
of the past... I see these recurring political, economic, and social trends as a means of looking forward into 
the future, while allowing us to avoid the mistakes of the past. Second, history teaches many lessons about 
the nature of human behavior, both past and present, providing insight into the actions, desires, and aspirations 
of those around me. Finally, it lays a solid foundation for several disciplines, including political science, 
economics, and international relations, three fields of great interest to me."""

In [89]:
testing = pd.DataFrame(data={'type': ['ENTP'], 'posts': [testPost]})

testPost, dummy  = text_preprocessing(testing, remove_stop_words=True, remove_mbti_profiles=True)
cnti= CVZ.transform(testPost)
tfid =  tfi.transform(cnti).toarray()

In [90]:
hyperpar = {}
hyperpar['n_estimators'] = 100
hyperpar['max_depth'] = 3
hyperpar['nthread'] = 7
hyperpar['learning_rate'] = 0.1

#XGBoost model for MBTI dataset
MyModelsPrediction = []

for i in range(len(personality_type)):
    M = personalityList[:,i]

    # Splitting the dataset into the relevant train and test data following the same 70 30 ratio 
    seed = 13
    test_size = 0.33
    X_train, X_test, y_train, y_test = train_test_split(A, M, test_size=test_size, random_state=seed)
    TypeModel = XGBClassifier(**hyperpar)
    TypeModel.fit(X_train, y_train)
    
    predictY = TypeModel.predict(tfid)
    MyModelsPrediction.append(predictY[0])

In [96]:
print("The Prediction Is ", convertBack(MyModelsPrediction))

The Prediction Is  JFES


#### References:

Towler, D. A. (n.d.). Why taking a personality test helps your professional development: CQ net. CQ Net - Management skills for everyone! Retrieved June 26, 2022, from https://www.ckju.net/en/blog/why-taking-personality-test-helps-your-professional-development 

Kamalesh, M. D., & B, B. (2022, March 11). Personality prediction model for social media using machine learning technique. Computers and Electrical Engineering. Retrieved June 26, 2022, from https://www.sciencedirect.com/science/article/abs/pii/S0045790622001446 

Tandera, T., Hendro, D., Suhartono, R., & Wongso, Y. L. (2017, October 13). Personality prediction system from Facebook users. Procedia Computer Science. Retrieved June 26, 2022, from https://www.sciencedirect.com/science/article/pii/S1877050917320537 

Christian, H., Suhartono, D., Chowanda, A., & Zamli, K. Z. (2021, May 17). Text based personality prediction from multiple social media data sources using pre-trained language model and model averaging - journal of big data. SpringerOpen. Retrieved June 26, 2022, from https://journalofbigdata.springeropen.com/articles/10.1186/s40537-021-00459-1 

Overview of personality prediction project using ML. GeeksforGeeks. (2020, September 5). Retrieved June 26, 2022, from https://www.geeksforgeeks.org/overview-of-personality-prediction-project-using-ml/ 

M, T., & K, M. A. (2021, June 4). Comparative study of personality prediction from social media by using machine learning and Deep Learning Method. International Journal of Engineering Research & Technology. Retrieved June 26, 2022, from https://www.ijert.org/comparative-study-of-personality-prediction-from-social-media-by-using-machine-learning-and-deep-learning-method 

Cherry, K. (2020, May 4). Why learning more about your personality type can benefit your life. Verywell Mind. Retrieved June 26, 2022, from https://www.verywellmind.com/reasons-to-learn-more-about-your-personality-type-4099388 

The advantages and disadvantages of personality tests. mettl. (2021, August 12). Retrieved June 26, 2022, from https://blog.mettl.com/advantages-disadvantages-of-personality-tests/ 

Yutuk, K. (2022, June 2). Pros and cons of personality tests. Aptitude.ph. Retrieved June 26, 2022, from https://www.aptitude.ph/benefits-of-personality-testing-for-companies/ 

f83lfV7Yd43Z. (2022, January 14). Can personality testing benefit your company? MidwestHR. Retrieved June 26, 2022, from https://www.midwesthr.com/personality-testing-benefits 

What is Project Design in project management? Wrike. (n.d.). Retrieved June 26, 2022, from https://www.wrike.com/project-management-guide/faq/what-is-project-design-in-project-management/ 

Scavetta, A. (2022, June 21). The best way to make a work plan. ProjectManager. Retrieved June 26, 2022, from https://www.projectmanager.com/blog/make-work-plan 

Project structure. Project Structure - an overview | ScienceDirect Topics. (n.d.). Retrieved June 26, 2022, from https://www.sciencedirect.com/topics/computer-science/project-structure#:~:text=A%20project%20structure%20provides%20the,of%20the%20local%20development%20landscape. 

Rajshreev. 2021. MBTI personality predictor using machine learning. (April 2021). Retrieved September 5, 2022 from https://www.kaggle.com/code/rajshreev/mbti-personality-predictor-using-machine-learning/notebook 